In [1]:
import sys
sys.path.append('../_amigocloud')

from sqlalchemy import create_engine, text
from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json
import pandas as pd

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [2]:
sys.path.append('..')

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [4]:
ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

In [5]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def cargar_link_to_db(url, cite):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.resumen_control_bio SET link = :url WHERE cite = :cite
            """)
            conn.execute(query, {"url": url, "cite": cite})
            print(f"✔️ Se registro link para CITE: {cite}" )
    except Exception as e:
        print(f"❌ Error al actualizar CITE: {cite}, {e}")

def get_registros_sin_enviar():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.resumen_control_bio
            WHERE enviado = false
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return pd.DataFrame()
    return None

In [6]:
convertir_docx_to_pdf()

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\10697_SLCB_16-10-2025_PAZ REA JAIME EDUARDO_CB-2025-157.docx
✅ Convertido: 10697_SLCB_16-10-2025_PAZ REA JAIME EDUARDO_CB-2025-157.docx
🗑️ Eliminado original: 10697_SLCB_16-10-2025_PAZ REA JAIME EDUARDO_CB-2025-157.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\10863_SLCB_16-10-2025_PAZ REA ROSENDO_CB-2025-159.docx
✅ Convertido: 10863_SLCB_16-10-2025_PAZ REA ROSENDO_CB-2025-159.docx
🗑️ Eliminado original: 10863_SLCB_16-10-2025_PAZ REA ROSENDO_CB-2025-159.docx


  0%|          | 0/1 [00:00<?, ?it/s]

...

C:\Users\Usuario\Desktop\PROYECTOS\_informes\8814_SLCB_13-10-2025_MURIALDO MENDEZ WALTER DANIEL_CB-2025-164.docx
✅ Convertido: 8814_SLCB_13-10-2025_MURIALDO MENDEZ WALTER DANIEL_CB-2025-164.docx
🗑️ Eliminado original: 8814_SLCB_13-10-2025_MURIALDO MENDEZ WALTER DANIEL_CB-2025-164.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\8838_SLCB_13-10-2025_MENDEZ PAZ MIRYAM_CB-2025-165.docx
✅ Convertido: 8838_SLCB_13-10-2025_MENDEZ PAZ MIRYAM_CB-2025-165.docx
🗑️ Eliminado original: 8838_SLCB_13-10-2025_MENDEZ PAZ MIRYAM_CB-2025-165.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\8842_SLCB_15-10-2025_MUÑOZ SUAREZ HUMBERTO_CB-2025-181.docx
✅ Convertido: 8842_SLCB_15-10-2025_MUÑOZ SUAREZ HUMBERTO_CB-2025-181.docx
🗑️ Eliminado original: 8842_SLCB_15-10-2025_MUÑOZ SUAREZ HUMBERTO_CB-2025-181.docx


In [7]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"✅[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"❌[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [8]:
contenido = os.listdir(directorio_destino)
print(len(contenido))
contenido

28


['10697_SLCB_16-10-2025_PAZ REA JAIME EDUARDO_CB-2025-157.pdf',
 '10863_SLCB_16-10-2025_PAZ REA ROSENDO_CB-2025-159.pdf',
 '1182_SLCB_15-10-2025_BOHORQUEZ FERNANDEZ NICANOR_CB-2025-172.pdf',
 '11860_SLCB_13-10-2025_ROCA SERRANO ARMANDO_CB-2025-160.pdf',
 '13638_SLCB_15-10-2025_TISCO VASQUEZ SERGIO_CB-2025-174.pdf',
 '1478_SLCB_15-10-2025_CALLE RALDEZ MARCO ANTONIO_CB-2025-176.pdf',
 '15085_SLCB_15-10-2025_CRUZ RUTH JAQUELIN B. DE_CB-2025-168.pdf',
 '15085_SLCB_15-10-2025_CRUZ RUTH JAQUELIN B. DE_CB-2025-170.pdf',
 '15195_SLCB_15-10-2025_VEIZAGA ENRIQUEZ GUSTAVO_CB-2025-175.pdf',
 '15195_SLCB_15-10-2025_VEIZAGA ENRIQUEZ GUSTAVO_CB-2025-177.pdf',
 '18189_SLCB_15-10-2025_BALCAZAR SOLETO RAMON BERTHY_CB-2025-179.pdf',
 '18189_SLCB_15-10-2025_BALCAZAR SOLETO RAMON BERTHY_CB-2025-180.pdf',
 '2353_SLCB_15-10-2025_CARO BENITES GERMAN_CB-2025-173.pdf',
 '2505_SLCB_15-10-2025_CRUZ BARRIENTOS CRISTIAN F._CB-2025-169.pdf',
 '31088_SLCB_14-10-2025_COLQUE QUIROZ JENNY FELICIDAD_CB-2025-167.pdf',
 '4

In [9]:
for file in contenido:
    partes = file.split('_')
    
    cite = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file
    id_drive = '1sEfhWGATb7tkKgr6gcknH3ZA_ed_hGTq'
    path_file = os.path.join(directorio_destino, file)
    print(cite)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    cargar_link_to_db(url_drive, cite)
    if os.path.exists(path_file):
        os.remove(path_file)
        print("✅ Archivo eliminado.")
    else:
        print("⚠️ El archivo no existe.")

CB-2025-157
[INFO] El archivo '10697_SLCB_16-10-2025_PAZ REA JAIME EDUARDO_CB-2025-157.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '10697_SLCB_16-10-2025_PAZ REA JAIME EDUARDO_CB-2025-157.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-157
✅ Archivo eliminado.
CB-2025-159
[INFO] El archivo '10863_SLCB_16-10-2025_PAZ REA ROSENDO_CB-2025-159.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '10863_SLCB_16-10-2025_PAZ REA ROSENDO_CB-2025-159.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-159
✅ Archivo eliminado.
CB-2025-172
[INFO] El archivo '1182_SLCB_15-10-2025_BOHORQUEZ FERNANDEZ NICANOR_CB-2025-172.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '1182_SLCB_15-10-2025_BOHORQUEZ FERNANDEZ NICANOR_CB-2025-172.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-172
✅ Archivo eliminado.
CB-2025-160
[INFO] El archivo '11860_SLCB_13-10-2025_ROCA SERRANO ARMANDO_CB-2025-160.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo '11860_SLC

In [21]:
import psycopg2

In [22]:
def obtener_conexion():
    return psycopg2.connect(
        host='localhost', 
        port=5433, 
        database='utea', 
        user='postgres', 
        password='A123456*')
    
def insertar_mensaje_whatsapp(cod_canero, nombre_canero, numero_contac, mensaje, enviado=False, fecha_enviado=None):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            INSERT INTO notificaciones_wsp.msj_whatsapp (
                cod_canero, nombre_canero, numero_cel, mensaje, enviado, fecha_enviado, numero_contac
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (cod_canero, nombre_canero, 0, mensaje, enviado, fecha_enviado, numero_contac))
        
        conexion.commit()
        cursor.close()
        print("Mensaje insertado correctamente.")
    except Exception as e:
        conexion.rollback()
        print("Error inesperado")
    return None

def marcar_resumen_lib_enviado(id_resumen):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            UPDATE drones_control_bio.resumen_control_bio
            SET enviado = true
            WHERE id = %s
        """, (id_resumen,))
        conexion.commit()
        cursor.close()
        print(f"Mensaje con id={id_resumen} marcado como enviado.")
    except Exception as e:
        conexion.rollback()
        print(f"Error inesperado: {e}")
    finally:
        if conexion:
            conexion.close()
    return None

In [34]:
PATH_XLSX_CONTAC=r'G:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - EQUIPO AVIACION UTEA\\Pulverizacion\\Data\\CONTACTOS.xlsx'

In [43]:
sin_enviar = get_registros_sin_enviar()

In [44]:
cods_ca = list(set(sin_enviar['id']))
print(cods_ca)

[97, 129, 105, 106, 109, 111, 113, 179, 180, 181, 116, 183, 118]


In [45]:
len(cods_ca)

13

In [46]:
df_contac = pd.read_excel(PATH_XLSX_CONTAC)
df_contac.fillna(0, inplace=True)
df_contac = df_contac[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [47]:
df_contac.head(3)

,cod_ca,nom_ca,telf01,telf02,telf03
0,0,CAÑERO TEST,78194371,0.0,0.0
1,86,AGUILERA TARADELLES JOSE LUIS,67759316,77010520.0,68848905.0
2,180,ELISEO AMURRIO,67559958,0.0,0.0


In [48]:
sin_enviar.head(3)

,id,fecha,institucion,cod_canero,nombre_canero,hectareas,cite,cod_propiedad,propiedad,producto,link,agrocittca,enviado
0,97,2025-09-24,8,1165,BEJARANO GANDARILLA ERWIN,11.03,CB-2025-90,40,SAN VICENTE--ERWIN BEJARANO,['TRICHOGRAMMA'],https://drive.google.com/file/d/1g8YpDinDp3gkD...,True,False
1,183,2025-10-15,304,1478,CALLE RALDEZ MARCO ANTONIO,3.53,CB-2025-176,1411,SAN JORGE--RENE CALLE,['TRICHOGRAMMA'],https://drive.google.com/file/d/1jh4DaMf1wyvzh...,True,False
2,181,2025-10-15,1,13638,TISCO VASQUEZ SERGIO,2.31,CB-2025-174,776,SINDICATO AGRARIO 2 DE FEBRERO(PARCELA-005)--T...,['TRICHOGRAMMA'],https://drive.google.com/file/d/12T1cZzglpUMm0...,True,False


### Mensaje dirigiendo a agrocittca

In [49]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'❌ Codigo: {cod} no tinene numeros validos registrados')
            continue
        
        for num in telefonos_validos:
            msj = f'''🌱 *Saludos {row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRÁ* para comunicarle que el informe del último servicio de *🦋 Control Biológico* realizado ya se encuentra disponible en *AgroCITTCA*.

📲 *Adjuntamos sus credenciales* para acceder a la App y los enlaces de descarga 👇

    👤 *Usuario:* {row['cod_canero']}
    🔑 *Contraseña:* 123*{row['cod_canero']}

⬇️ *Descargue la App:* 

📱 *Android:* https://play.google.com/store/apps/details?id=com.dima.guabira  

🍎 *iPhone:* https://apps.apple.com/us/app/agro-cittca-guabir%C3%A1/id1669149924

🚀 Una vez instalada la app, inicie sesión con su usuario y contraseña, y acceda a:  
📂 *AGROCITTCA → Seguimiento a campo → Informes*

Si tiene inconvenientes para acceder o visualizar los informes, no dude en consultarnos. ¡Gracias por su confianza! 🙌
'''         
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 1165 - BEJARANO GANDARILLA ERWIN no tiene registro telefonico
Codigo: 1478 - CALLE RALDEZ MARCO ANTONIO no tiene registro telefonico
Codigo: 13638 - TISCO VASQUEZ SERGIO no tiene registro telefonico
Codigo: 1182 - BOHORQUEZ FERNANDEZ NICANOR no tiene registro telefonico
Codigo: 2353 - CARO BENITES GERMAN no tiene registro telefonico
Codigo: 16213 - ZAPATA SERON SIMON no tiene registro telefonico
Codigo: 2351 - CANDIA RUIZ ADALBERTO no tiene registro telefonico
Codigo: 18062 - MERCADO MERCADO RUBEN DARIO no tiene registro telefonico
Codigo: 13187 - SUAREZ CUELLAR REMBERTO no tiene registro telefonico
Codigo: 2565 - CANDIA RUIZ JOSE LUIS no tiene registro telefonico
Codigo: 42229 - CANDIA ROJAS JOSE LUIS no tiene registro telefonico
Codigo: 42378 - CANDIA RUIZ ORLANDO no tiene registro telefonico
Codigo: 41562 - SUAREZ SALAS ERWIN RONALD no tiene registro telefonico


### Mensaje dirigiendo a PDF

In [24]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'Codigo: {cod} no tinene numeros validos registrados')
            continue
        
        for num in telefonos_validos:
            msj = f"Saludos *{row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRA* para hacer llegar una copia del informe de servicio de control biologico en caña de azucar. También informale este informe lo puede encotrar en la aplicación AgroCITTCA, y que a partir de los proximos servicos los informes se enviaran direntamente a AgroCITTCA. Adjunto informe: {row['link']}"
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 1165 - BEJARANO GANDARILLA ERWIN no tiene registro telefonico
Codigo: 16213 - ZAPATA SERON SIMON no tiene registro telefonico
Codigo: 2351 - CANDIA RUIZ ADALBERTO no tiene registro telefonico
Codigo: 18062 - MERCADO MERCADO RUBEN DARIO no tiene registro telefonico
Codigo: 13187 - SUAREZ CUELLAR REMBERTO no tiene registro telefonico
Codigo: 2565 - CANDIA RUIZ JOSE LUIS no tiene registro telefonico
Codigo: 42229 - CANDIA ROJAS JOSE LUIS no tiene registro telefonico
Codigo: 42378 - CANDIA RUIZ ORLANDO no tiene registro telefonico
Codigo: 41562 - SUAREZ SALAS ERWIN RONALD no tiene registro telefonico
